#explanatory



# Inequality simulator

This code walks you through the *Inequality Simulator*. This is not meant to information policy, but to educatate individuals on the economphysics of wealth distribution. 

The challenge is what are the right policies to opitmize the overall health of the economy. 

#explanatory

Code is stored knowledge

Sharing tha tknowledge is why the open source ecosystem is so powerful. 

This cell imports free libraries. 

#TODO Add links

Numpy - numerical python, a library that helps us manage the wealth details we use
Scipy - scientific python, - a library that helps us create known innovation and wealth distribution phenomenon
Solara - a libaray that create visual interface for the simulation
Matplotlib - A library that creates visuals 
Mesa - a simulation library that we use to build this inequality simulator



In [1]:
import numpy as np
from scipy.stats import expon
import solara 
from matplotlib.figure import Figure

import mesa
from mesa.visualization.utils import update_counter
from mesa.visualization import SolaraViz, make_plot_component


<IPython.core.display.Javascript object>

## Helper Functions

We need to functions to help us assess our simulation

**calculate churn** This function tells us the social mobility of our population. Do the rich always stay rich and do the poor always stay poor? Or, do our policies allows us to indivuals to move economicaslly based on the value of their ideas> 

**number to words** THis function simply makes it easier to understand how much money our simualted ecnomy is generating. 


### Calculate Churn

This function goes through each agent in the model and determines if an agent moved up an economic class or down. 

All agents start with 1 unit of wealth and so all start in the "middle class"

As each "step" occurs in the simulation, the model tracks how many move betwen classes and so change form their previous class to a new class. 

#explanatory

The functions first set to zero variables of moving up or down. 

Next the function goes through each agents and asks if the agents brackets changes (the bracket are determined futher one) 

Then the model simply compares the previous economic income bracket with its current income bracket. If the income bracket changed the function then adds a plus one to whether the agent moved up or down.   




In [2]:
def calculate_churn(model): 
    #zero variables
    moving_up = 0
    moving_down = 0

    # Iterate through each agent
    for agent in model.agents: 
        # Determine if something changed
        if agent.previous != agent.bracket: 
            if agent.previous == "Middle":
                if agent.bracket == "Upper":
                    moving_up += 1
                elif agent.bracket == "Lower":
                    moving_down += 1
            if agent.previous == "Upper":
                if agent.bracket == "Middle" or agent.bracket == "Lower": 
                    moving_down += 1
            if agent.previous == "Lower":
                if agent.bracket == "Upper" or agent.bracket == "Middle":
                    moving_up += 1
    totals = model.agents.groupby("bracket").count()
    if "Upper" not in totals:
        totals["Upper"] = 0
    if "Middle" not in totals:
        totals["Middle"]=0
    if "Lower" not in totals: 
        totals["Lower"]=0
    return [moving_up, moving_down, totals]

### Numbers to Words

This function simply tasks the total amount of wealth being created in the model and make it more easily readable. 

#explanatory

This function writes out thresholds based on how much wealth is in the economy. 

If the threshhold is above a billion it takes that 1,000,000,000 and turns it to a billion.  

In [3]:
def number_to_words(n):
    # Define thresholds and corresponding words, adding "Trillion"
    thresholds = [
        (1_000_000_000_000, 'Trillion'),
        (1_000_000_000, 'Billion'),
        (1_000_000, 'Million'),
        (1_000, 'Thousand')
    ]
    
    # Loop through thresholds to determine appropriate scale
    for threshold, word in thresholds:
        if n >= threshold:
            value = n / threshold
            return f"{int(value)} {word}"
    
    # If less than 1000, just return the number as a string
    return str(n)   
    

## Visuals

This section creates the visuals we use for the simulation. It creates 4 visuals. 

1. Histogram of the wealth, this shows the number of people (agents) (y -axis) and the amount of wealth (x-axis)
2. Churn visual shows how many people are changing economic class
3. Gini Coefficient shows a standard measure of wealth distribution. If one person has all the wealth the gini coeeficient is 1, if the each person has the exact same amount of wealth the gini coefficient is 0
4. The total wealth is the welath of each agent added up to create thte total wealth of the population

### Histogram

This function plots a histogram of the wealth distribution of the agents. 

A histogram takes the number of of agents with wealth between some value such as 0 and 10 and then counts them. 

In this simulation we have 200 agents. There may be 80 wealth wealth between 0 and 10, and then 10 with wealth between 10 and 50, and then 5 with wealth between 50 and 100 and then 5 more with wealth at different levels that puts each agent in their own bin. 

#explanatory

This function uses matplolib and grpahing library we imported earlier and in one line takes a list of the agent wealth and turns it into a histogram. 

Breaking down this function we have: 
- update_counter this is a Mesa (imported library) function that get all the informatin frm the simulation
- figure and ax create the base graph
- wealth_vals gets the wealth of all 200 agents and puts it in a list so [1, 40, 200 ... etc]
- ax.hist places those values in a histogram
- solara.FigureMatplib the plots this data in a grpah in our simulation

In [4]:
@solara.component
def Histogram(model):
    update_counter.get() # This is required to update the counter
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    wealth_vals = [agent.wealth for agent in model.agents]
    # Note: you have to use Matplotlib's OOP API instead of plt.hist
    # because plt.hist is not thread-safe.
    ax.hist(wealth_vals, bins=10)
    solara.FigureMatplotlib(fig)

### Churn

This function plots the number of agent sin each ecnomic class and the movement of agents between classes.

#explanatory

The update_count.get is a mesa function to get the data form the simulation

The figure and ax are matplotlib functions to create the base plot on a cartesian system

The churn details is the result of the `calculate_churn` function discussed previously

Each of the next lines plots the class or movement with the first two numbers being their x and y location, then what words to plot, and finally some style such as color and font size.

Finally the Solara plot function so it show up on the dashboard

In [5]:
@solara.component
def Churn(model): 
    update_counter.get()

    fig = Figure()
    ax = fig.subplots()
    ax.axis('off')
    churn_details = calculate_churn(model)
    ax.text(0.2, 0.8, f"Upper Class: {round((churn_details[2]["Upper"]/200)*100,0)}% ", va='center', ha='left', color="green", fontsize=15)
    ax.text(0.2,0.5,f"Middle Class {round((churn_details[2]["Middle"]/200)*100,0)}% ",  va='center', ha='left', color="orange", fontsize=15)
    ax.text(0.2, 0.2, f"Lower Class {round((churn_details[2]["Lower"]/200)*100,0)}% ", va='center', ha='left', color="red", fontsize=15)
    ax.text(0.6, 0.6, f"Moving Up {round((churn_details[0]/200)*100,2)}%", va='center', ha='left', color="black", fontsize=15)
    ax.text(0.6, 0.4, f"Moving Down {round((churn_details[1]/200)*100,2)}%", va='center', ha='left', color="black", fontsize=15)
    
    solara.FigureMatplotlib(fig)

### Gini Coefficient

[The Gini coefficient is a measure of wealth distribution](https://en.wikipedia.org/wiki/Gini_coefficient)

If one agent has all the wealth the number will be 1. If the wealth is perfectly distirbuted where all agents have exactly the same amount the wealth will be 0. 

[Here is the World Bank's list of Gini Coefficients by country](https://data.worldbank.org/indicator/SI.POV.GINI)

#explanatory

This function first gets a list of the wealth of all agents

In then sorts them from poorest to richest

It then takes the agent wealth, multiplied by the agents rank and then normalized for the wealth of the population. 

For the return discounting the 1/200 if B equals 0.5, this means perfect equity, becuase you will have 1-(2 * 0.5). If B equals 0 then you have complete inequity. 

Finally it returns gini coefficient. 

In [6]:
def compute_gini(model):
    agent_wealths = [abs(float(agent.wealth)) for agent in model.agents]
    x = sorted(agent_wealths)
    N = model.population
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B

### Population Wealth

Based on the policy the simulation is running, the total welath of the population will change. This visuals simply shows how much total welath isin the population 

#explanatory

- update_counter gets the most recent data form the model
- fig and ax create the based graph
- total_wealth adds of the wealth of each agent to the total wealth
- words calls the earlier discussed number_to_words function and converts the very large numbers (in some cases) to a more readble format
- the ax.text plot the policy, and total wealth
- the solara line plots it on the dashboard

In [7]:
@solara.component
def Wealth(model): 
    update_counter.get()
    fig = Figure()
    ax = fig.subplots()
    ax.axis('off')
    total_wealth = sum([agent.wealth for agent in model.agents])
    words = number_to_words(int(total_wealth))
    ax.text(0.0,0.8,f"Total Wealth Using {model.policy} Policy", 
             va='center', ha='left', color="black", fontsize=20)
    ax.text(0.5,0.5,f"{words}", 
             va='center', ha='left', color="black", fontsize=20)
    solara.FigureMatplotlib(fig)



## Simulation

The next two sections create the model. There is really two parts:

- Agent: creates the agents and there inateraction logic
- Model: the manager class that creates the agents, also manages their interaction, and collects data

### Agent

This class determines the agent behaviors

The logic is fairly straightforward. 
- Agents receive a payday based on their W attribute
- Agents then pay a survival fee which adjusts for inflation (#Should this be deleted?)
- They then pay another agent which is the econophysics aspect for the exchange of goods and services
If there is a powerful leader policy or equal distribution policy there is then a party elite:
- In this case agents pay a tax to the party elite
If there is an innovation policy:
- Agents can increase their wealth (or decrease if they have a bad idea) if they have the start up capital
- If they have start up capital then the agent pay is adjust based on the quality of their innovation
- The benefit of the innovation then decays over time
Finally, the agent economic class is updated

#explanatory

The __init__ function initializes the agent with several attributes (many of these are determined in the model class next):
- wealth: Starts at 1, representing the agent's initial wealth.
- party_elite: True or False indicating if the agent is part of the elite group.
- bracket: Indicates the wealth bracket ("Lower", "Middle", or "Upper").
- W: Proportion, used to calculate wealth growth.
- I: Innovation factor that influences the agent's wealth growth over time.
- ioriginal: Stores the initial innovation factor for resetting later.
- decay: Tracks how the innovation factor decreases over time.

**Payday** The agent's wealth increases proportionally (W) to its current wealth.

**Survivial costs**
Adjusts the agent's wealth for survival costs (e.g., inflation). If the agent's wealth is less than or equal to the survival cost, it loses all wealth.

**Econophysics**

Simulates a wealth exchange between the agent and a randomly chosen peer.
Transfers a portion of the agent's wealth (exchange_agent.W * self.wealth) to the other agent, creating a dynamic wealth redistribution mechanics

**Powerful leader or wealth distribution**

If the policy supports "powerful leaders" or "equal wealth distribution," the agent pays a 5% tax to a randomly chosen party elite.

**Innovation**

Innovation policy increases the wealth growth rate (W) by multiplying it with the innovation factor (I) if the agent's wealth exceeds the initial capital.
Innovation decreases over time due to decay, which starts at 0 and increases with each step.
If the wealth drops below the initial capital, the decay resets, and the innovation factor reverts to its original value.

In [12]:
class WealthAgent(mesa.Agent):
    
    def __init__(self,model, proportion,innovation,party_elite):
        super().__init__(model)
        self.wealth=1
        self.party_elite = party_elite
        self.bracket = "Middle"
        self.previous = "Middle"
        self.W = proportion
        self.I = innovation
        self.ioriginal = innovation
        self.decay = 0
        
    
    def step(self):
        
        """
                                PAYDAY
        """
        self.previous = self.bracket
        #increase wealth by proportion - payday
        self.wealth += (self.W*self.wealth)
        
        
        #Survivial cost which adjust for total wealth in the model (e.g. inflation)-- Should this be deleted
        if self.wealth > self.model.survival_cost and self.wealth > 0: 
            self.wealth -= self.model.survival_cost
        else: 
            self.wealth -= self.wealth
        
        """
                                   Econophysics core reasoning
        """
                        
        exchange_agent = self.random.choice(self.model.agents)
        
        if self.wealth >= 0 and exchange_agent is not None and exchange_agent is not self:
            exchange_agent.wealth += (exchange_agent.W*self.wealth)
            self.wealth -= (exchange_agent.W*self.wealth)      
        
        '''
                                Powerful Leaders or Equal Wealth
        '''
        
        #If powerful leaders or equal wealth agents have to pay a tax to the party elites
        if self.model.policy=="powerful leaders" or self.model.policy=="equal wealth distribution": 
            party_elites = self.model.agents.select(lambda a: a.party_elite==True)
            #pay tax to the party_elite
            party_elite = self.random.choice(party_elites)
            party_elite.wealth += self.wealth*.05
            self.wealth -= self.wealth*.05
        
        """
                            INNOVATION
        """
           
        if self.model.policy=="Innovation": 
            if self.wealth > self.model.inital_captial: 
                #increase payday by innovation
                self.W*=self.I
                #Value of innovation decreases over time
                self.I-=self.decay #starts at 0
                #increase decay for next step 
                self.decay+=0.1
            else: 
                self.decay = 0 
                self.I = self.ioriginal

        #calculate bracket
        if self.wealth < self.model.brackets[0]:
            self.bracket = "Lower"
        elif self.wealth >= self.model.brackets[1]:
            self.bracket = "Upper"
        else: 
            self.bracket = "Middle"
       

### Model

The class provides the management for the simulation. It starts with two helper functions, calculating the economic classes (needs a better approach) and then calculating the start up funding necessary to invest in innovation

Next is the model class which first creates each agent and assigns them their attributes based on statiscial distributions. 

Finally is the step function which calls on the agents and executes the logic of the various policies. 

#explanatory

`calc_brackets` Function
- **Purpose**: This function calculates wealth brackets for the agents in the model. It divides the agents into three categories: "Lower," "Middle," and "Upper" wealth brackets.
- **How it works**: It identifies the maximum wealth among all agents and calculates thresholds at 33% and 67% of this maximum value. These thresholds define the boundaries for each bracket.



`start_up_required` Function
- **Purpose**: Determines the startup capital required for innovation based on the wealth distribution among agents.
- **How it works**: It uses Sturges' rule to calculate the number of bins for dividing wealth. Then, it identifies the maximum wealth in each bin. This allows the model to assess how wealth is distributed and determine the required capital for innovation based on the selected policy.



`WealthModel` Class

`__init__` Method
- **Purpose**: Initializes the model and its key attributes.
- **Attributes**:
  - `policy`: Specifies the policy being simulated (e.g., "econophysics," "powerful leaders," "equal wealth distribution," or "innovation").
  - `population`: Defines the number of agents in the simulation.
  - `brackets`: Initial thresholds for categorizing agents into wealth brackets.
  - `start_up_required`: Configures how startup capital is determined for innovation.
  - `datacollector`: Tracks key metrics like Gini coefficient, total wealth, and individual agent attributes over time.
- **Agent Initialization**: 
  - Agents are initialized with a wealth proportion (`W`) from a normal distribution and an innovation factor (`I`) from a Pareto distribution. 
  - Party elites are identified as the top 5% of agents based on wealth.
`step` Method
**Purpose**: Defines the model's behavior at each time step. It updates wealth brackets, calculates survival costs, and implements the selected policy.


Key Components:
1. **Calculating Brackets**:
   - Updates wealth thresholds dynamically based on the current maximum wealth in the system.

2. **Survival Costs**:
   - Determines a survival cost (e.g., inflation) as the 10th percentile of an exponential distribution scaled to the mean wealth.

3. **Policy Implementation**:
   - **Econophysics**: Agents exchange wealth randomly with others, simulating a redistribution mechanism.
   - **Powerful Leaders**: Non-elite agents interact and transfer wealth, but elites are exempt from giving.
   - **Equal Wealth Distribution**: Wealth is redistributed equally among all agents at the start of each time step.
   - **Innovation**: Startup capital for innovation is determined based on wealth distribution, and agents with sufficient capital experience accelerated growth.

4. **Data Collection**:
   - Collects data every five steps to capture key metrics for analysis while avoiding excessive noise in visualizations.

In [20]:
def total_wealth(model): 
    return sum([agent.wealth for agent in model.agents])

def calc_brackets(model):  
        most = max([agent.wealth for agent in model.agents])
        return [int(most*0.33), int(most*0.67)]

def start_up_required(model): 

    # Number of bins using Sturges' rule
    num_bins = int(np.ceil(np.log2(model.population) + 1))
    
    wealth_list = np.array([agent.wealth for agent in model.agents])
    # Create the bins
    bin_edges = np.linspace(min(wealth_list), max(wealth_list), num_bins + 1)
    
    # Find the max value in each bin
    bin_max_values = []
    for i in range(len(bin_edges) - 1):
        # Get the lower and upper bound of the current bin
        lower_bound = bin_edges[i]
        upper_bound = bin_edges[i + 1]
        # Find the values in this bin
        values_in_bin = wealth_list[(wealth_list >= lower_bound) & (wealth_list < upper_bound)]
        
        # Find the max value in the bin (if any values are in the bin)
        if len(values_in_bin) > 0:
            bin_max_values.append(max(values_in_bin))
        else:
            bin_max_values.append(None)  # No values in this bin
    return bin_max_values
    
class WealthModel(mesa.Model): 
    
    def __init__(self, policy="econophysics", population=200, start_up_required = 1, seed=42):
        
        super().__init__()
        self.policy = policy
        self.population = population
        self.party_elite = None
        self.total = 200
        self.survival_cost = 0
        self.each_wealth = 0
        self.brackets = [0.75,1.25]
        self.start_up_required = start_up_required
        self.inital_capital = 1.5
        
        self.datacollector = mesa.DataCollector(model_reporters = {"Gini": compute_gini,"Total": total_wealth },
                                               agent_reporters={"Wealth":"wealth", "Bracket":"bracket",
                                                                "Pay":"W", "Innovation":"I" })
        
        
        mean = 0.2      # mean of the distribution
        sigma = 0.05     # original sigma, so variance is 2*sigma^2
        variance = 2 * sigma**2

        # Generate 200 data points from the Gaussian distribution
        payday_array = np.random.normal(mean, np.sqrt(variance), self.population)
        
        
        innovation_array =  np.random.pareto(2.5,size=200)
        '''
        innovation_array = np.random.normal(loc=1,
                                          scale=0.1,
                                          size=self.population)
        '''
        # round array to two decimals
        payday_array = np.around(payday_array, decimals=2)
        party_elite_cut= np.percentile(payday_array, 95)
        
        innovation_array = np.around(innovation_array, decimals=2)
        
        for idx in range(self.population):
            party_elite=False
            if payday_array[idx] >=party_elite_cut: 
                party_elite=True
            WealthAgent(self, float(payday_array[idx]), float(innovation_array[idx]), party_elite)
    
    def step(self):
        self.brackets = calc_brackets(self)
        self.total = total_wealth(self)
        #Deternine survival cost based on inflation
        exp_scale = np.mean([agent.wealth for agent in self.agents])
        self.survival_cost = expon.ppf(0.1, scale=exp_scale)
        if self.policy=="econophysics": 
            self.agents.shuffle_do("step")
        
        # party_elites can only receive from subordinates but never give money
        if self.policy=="powerful leaders": 
            subordinates = model.agents.select(lambda a: a.party_elite==False)
            subordinates.shuffle_do("step")
        
        # Divide the wealth equally among all agents at the beginning of the time step
        if self.policy=="equal wealth distribution": #Should this exlcude the party elite
            each_wealth = self.total/self.population
            for agent in self.agents: 
                agent.wealth=each_wealth
            self.agents.shuffle_do("step")

        if self.policy=="innovation": 
            bins = start_up_required(self)
            if self.start_up_required==1: 
                self.initial_capital = bins[0]
            elif self.start_up_required==2: 
                mid = len(bins) // 2
                self.initial_capital =bins[mid]
            else:
                self.initial_capital =bins[-1]
            self.agents.shuffle_do("step")
             
        if self.steps//5 == 0: #Only collect every 5 steps to make visuals more understandable 
            self.datacollector.collect(self)
       
       
        
       

## Dashboard

This cell create the interactive dashboard for the model 

In [21]:
model = WealthModel()
model_params = {
    "policy": {
        "type": "Select",
        "value": "econophysics",
        "values": ["econophysics", "powerful leaders","equal wealth distribution", "innovation"],
        "text": "Select Policy"
    }
}

wealth_plot = make_plot_component("Gini") #add labels and titles

dash = SolaraViz(
    model, 
    components=[Churn, wealth_plot, Wealth, Histogram],
    model_params=model_params,
)

dash

Cannot show ipywidgets in text